## Performing HPT ##

In [3]:
from model import VesselSegmentationModel
import wandb
import torch    
import pytorch_lightning as pl

In [4]:
torch.cuda.empty_cache()

In [2]:
wandb.login(relogin=True, verify=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/paperspace/.netrc


True

In [3]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'f1_score',
        'goal': 'maximize',
    },
    'parameters': {
        'batch_size': {
            'distribution': 'q_log_uniform_values',
            'q': 4,
            'min': 8,
            'max': 64,
        },
        'lr': {
            'distribution': 'uniform',
            'min': 1e-5,
            'max': 5e-3,
        },

        'lambda_kl': {
            'distribution': 'uniform',
            'min': 0.05,
            'max': 0.2,
        },

        'lambda_l2': {
            'values': [1e-1, 5e-2, 1e-4],
        },

        'lambda_bce': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.5,
        },

        'lambda_dice': {
            'distribution': 'normal',
            'mu': 1.0,
            'sigma': 0.4,
        },

        'lambda_tversky': {
            'values': [0.0, 0.1, 0.5, 1.0],
        },

        'lambda_label': {
            'values': [0.0, 0.5, 1e1],
        },

        'lambda_focal': {
            'values': [0.0, 0.1, 0.5, 1.0],
        },

        'drop_enc': {
            'distribution': 'uniform',
            'min': 0.0,
            'max': 0.7,
        },
        'drop_label': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.7,
        },
    }
}

In [4]:
sweep_id = wandb.sweep(sweep_config, project="DMI-2024")
wandb_logger = pl.loggers.WandbLogger()

Create sweep with ID: ghb5zu78
Sweep URL: https://wandb.ai/nlp-course-zvi-and-tomer/DMI-2024/sweeps/ghb5zu78


In [5]:
with wandb.init() as run:
    config = run.config
    run_id = run.id
    model = VesselSegmentationModel(config)
    callbacks = [
        pl.callbacks.ModelCheckpoint(
            monitor="val_dice_loss",
            dirpath=f"./models/{run_id}",
            save_top_k=3,
            mode="min",
        ),
        pl.callbacks.LearningRateMonitor(logging_interval="step"),
    ]
    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="auto",
        callbacks=callbacks,
        log_every_n_steps=1,
        enable_progress_bar=True,
        logger=wandb_logger,
    )
    trainer.fit(model)
    trainer.test(model)
    wandb.log(model.metrics)
    wandb.log(model.hparams)
    wandb.finish()

wandb: Currently logged in as: zvi-badash (nlp-course-zvi-and-tomer). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/paperspace/DMI2024B/.venv/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | UNet | 11.3 M | train
---------------------------------------
11.3 M    Trainable params
0         Non-train

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/home/paperspace/DMI2024B/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]